In [964]:
import os
import pandas as pd 
from glob import glob

import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re

import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
import html

from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Twitter, Mecab,Kkma
from konlpy.utils import pprint

import chat_bot as cb
import Database_Handler as dh

ct = ctwitter()
ot = Twitter()
mecab = Mecab()
xxxx = ct._dictionary._pos2words
print (list(map(lambda x: len(xxxx[x]), xxxx)))

score_weights = {
    'num_nouns': -0.1,
    'num_words': -0.2,
    'no_noun': -1
}

def my_score(candidate):
    num_nouns = len([w for w,t in candidate if t == 'Noun'])
    num_words = len(candidate)
    no_noun = 1 if num_nouns == 0 else 0
    
    score = (num_nouns * score_weights['num_nouns'] 
             + num_words * score_weights['num_words']
             + no_noun * score_weights['no_noun'])
    return score

ct.set_selector(score_weights, my_score)

[512, 1464079, 5532]
[('이', 'Noun'), ('것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]
best: [('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]


In [965]:
def Check(list1, list2):
    first_keywords = list(filter(lambda x: x[0] == x[1], list(itertools.product(list1, list2))))
    first_keywords = set(map(lambda x: x[0], first_keywords))
    second_keywords = list(filter(lambda x: x[0] != x[1], list(itertools.product(list1, list2))))
    second_keywords = list(filter(lambda x: abs(len(max(x, key = len)) - len(''.join(re.findall('['+x[0]+']', x[1])).strip())) == 1,
                                  second_keywords))
    second_keywords = set(map(lambda x: max(x, key = len), second_keywords))
    keywords = first_keywords|second_keywords
    k2 = list(filter(lambda idx: abs(len(''.join(re.findall('['+idx[0]+']',idx[1])).strip())-len(min(idx, key = len))) == 0,
                     list(itertools.combinations(keywords, 2))))
    k2 = set(map(lambda x: min(x, key = len), k2))
    list(map(lambda x: keywords.remove(x), k2))
    return keywords

def CompareWords(word1, word2):
    if len(word1) == len(word2):
        return word1, word2
    else:
        return min([word1, word2], key = len), max([word1, word2], key = len)

def Check2(list1, list2):
    first_keywords = list(filter(lambda x: x[0] == x[1], list(itertools.product(list1, list2))))
    first_keywords = set(map(lambda x: x[0], first_keywords))
    second_keywords = list(filter(lambda x: x[0] != x[1], list(itertools.product(list1, list2))))
    second_keywords = list(filter(lambda x: abs(len(CompareWords(*x)[1]) - len(''.join(re.findall('('+CompareWords(*x)[0]+')?', CompareWords(*x)[1])).strip())) == 1,
                                  second_keywords))
    second_keywords = set(map(lambda x: CompareWords(*x)[1], second_keywords))
    keywords = first_keywords|second_keywords
    k2 = list(filter(lambda idx: abs(len(''.join(re.findall('['+CompareWords(*idx)[1]+']',CompareWords(*idx)[0])).strip())-len(min(idx, key = len))) == 0,
                     list(itertools.combinations(keywords, 2))))
    k2 = set(map(lambda x: min(x, key = len), k2))
    list(map(lambda x: keywords.remove(x), k2))
    return keywords

def Compare_Keywords(list1, list2, list3):
    list1 = list(map(lambda x: ' '.join(list(dict.fromkeys(x.split(' ')))).strip(), list1))
    list1 = list(dict.fromkeys(list1 + list(map(lambda y: ''.join(y), list(map(lambda x: x.split(' '), list1))))))
    list2 = list(map(lambda x: ' '.join(list(dict.fromkeys(x.split(' ')))).strip(), list2))
    list2 = list(dict.fromkeys(list2 + list(map(lambda y: ''.join(y), list(map(lambda x: x.split(' '), list2))))))
    list3 = list(map(lambda x: ' '.join(list(dict.fromkeys(x.split(' ')))).strip(), list3))
    list3 = list(dict.fromkeys(list3 + list(map(lambda y: ''.join(y), list(map(lambda x: x.split(' '), list3))))))
    m1 = Check2(Check(list1, list2), Check(list2, list1))
    m2 = Check2(Check(list1, list3), Check(list3, list1))
    m3 = Check2(Check(list2, list3), Check(list3, list2))
    total =m1 | m2 | m3
    xyz2 = list(itertools.combinations(total, 2))
    xyz3 = list(filter(lambda x: abs(len(''.join(re.findall('('+CompareWords(*x)[0]+')',CompareWords(*x)[1])).strip()) - len(CompareWords(*x)[0]))==0, xyz2))
    xyz3_1 = set(map(lambda x:CompareWords(*x), xyz3))
    xyz3 = set(map(lambda x:CompareWords(*x)[0], xyz3))
    xyz3 = set(filter(lambda x: x!='', xyz3))
    list(map(lambda x: total.remove(x), xyz3))
    xyz2 = list(itertools.combinations(total, 2))
    xyz4 = list(filter(lambda x: abs(len(''.join(re.findall('['+CompareWords(*x)[0]+']',CompareWords(*x)[1])).strip()) - len(CompareWords(*x)[0]))==0, xyz2))
    xyz4_1 = set(map(lambda x: CompareWords(*x), xyz4))
    xyz4 = set(map(lambda x: CompareWords(*x)[0], xyz4))
    xyz4 = set(filter(lambda x:x!='', xyz4))
    list(map(lambda x: total.remove(x), xyz4))
    xyz2 = list(itertools.combinations(total, 2))
    xyz5 = set(filter(lambda x: re.findall('('+CompareWords(*x)[0]+')',CompareWords(*x)[1]),xyz2)) 
    lstCheck = list(map(lambda x: (CompareWords(*x)[1], xxx(*x)), xyz3_1 | xyz4_1 | xyz5))
    lstCheck = list(filter(lambda x: x[1]!=None, lstCheck))
    total = list(total)
    if lstCheck:
        for check in lstCheck:
            total[total.index(check[0])] = check[1]
    return set(total)

    

def xxx(x1,x2):
    lessWord = CompareWords(x1,x2)[0]
    moreWord = CompareWords(x1,x2)[1]
    out1 = re.findall(lessWord, moreWord)
    if len(out1) > 1:
        return out1[0]
    
    elif len(out1) == 1:
        if re.search(lessWord, moreWord):
            out2 = re.sub(re.search(lessWord, moreWord).group(), '', moreWord).strip()
            lessWord2 = CompareWords(out2, lessWord)[0]
            moreWord2 = CompareWords(out2, lessWord)[1]
            if re.search(lessWord2, moreWord2):
                return moreWord2
            else:
                return None
        else:
            return None
    else:
        return None

In [966]:
ext_Keyword_Daum_mecab = glob('./data/Keyword_outcome/mecab-daum*')
ext_Keyword_Daum_ckonlpy = glob('./data/Keyword_outcome//ckonlpy-daum*')
ext_Keyword_Daum_both = glob('./data/Keyword_outcome/mecab_ckonlpy-daum*')

ckonlpyDict = dict()
for file in ext_Keyword_Daum_ckonlpy:
    ckonlpyDict.update(pickle.load(open(file,'rb')))
mecabDict = dict()
for file in ext_Keyword_Daum_mecab:
    mecabDict.update(pickle.load(open(file,'rb')))
bothDict = dict()
for file in ext_Keyword_Daum_both:
    bothDict.update(pickle.load(open(file,'rb')))

In [967]:
#site = 'Naver'
site = 'daum'
if site == 'daum':
    collection = 'newsDaum'
elif site.lower() == 'naver':
    collection = 'newsNaver'
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
useCollection = dh.Use_Collection(useDb, collection)

In [968]:
dataList = useCollection.find({'site':site})
i = 0 
for idx, data in enumerate(dataList):
    print (idx, data['title'])
    print (data['link'])
    print (data['keywords'])
    idIs = data['_id']._ObjectId__id.hex()
    if idIs in ckonlpyDict.keys() and idIs in mecabDict.keys() and idIs in bothDict.keys():
        keywords = Compare_Keywords(ckonlpyDict[idIs], mecabDict[idIs], bothDict[idIs])
        print (keywords)

0 김국진·노홍철·한혜진 '한명회', 오늘(5일) 9회만에 종영
http://v.media.daum.net/v/20171205151743503
['노홍철', '한명회', '한혜진', '종영', '편성시간']
{'녹화', '방송', '이름을', '특집', '한명회 9회', '출연자들'}
1 '몸 만드는 게 우선' 류중일 감독, 캠프 명단에 신인 없다
http://v.media.daum.net/v/20171209104014068
['류중일']
{'선수들', '류중일 감독', '오키나와 캠프', '신인', '명단'}
2 [어제TV]첫방 '저글러스' 男보스 女비서, 시대착오적 설정 '불편'
http://v.media.daum.net/v/20171205061623329
['불륜', '비', '첫방', '최다니엘', '최대철']
{'불륜', '설정', '보스', '비서', '봉상무', '좌윤이', '상무 아내', '무릎'}
3 [Oh! 무비] '기억의 밤' 장항준 감독은 어떻게 골리앗을 무찔렀나
http://v.media.daum.net/v/20171205140114189
['기억의밤', '시나리오', '리얼리티', '강하늘', '김무열']
{'기억의', '영화', '드라마', '장항준 감독', '시나리오', '스릴러'}
4 [RE:TV]'섬총사' 강호동X정상훈, 80cm 대어 낚아 '낚시TV 따로없네'
http://v.media.daum.net/v/20171205065011570
['강호동', '정상훈', '조세호', '어청도', '달타냥']
{'낚시', '강호동과 정상훈', '섬총사', '어청도', '조세호', '농어'}
5 '범죄도시'는 왜 불법 다운과의 전쟁을 선포했나
http://v.media.daum.net/v/20171204154759778
['윤계상', '마동석', '강경대응', 'sns']
{'범죄 도시', '영화', '불법', '청소년', '유포자들', '관람 불가', '행위'}
6 '다둥이 아빠' 박지헌, 여섯째 출산 앞둔 아내에 "정말 고마워" 애틋

{'인사', '상태', '부진', '각오', '신혼 여행', '최대성', '모습을', '트레이드', '두산'}
51 "사장이 저를 '개XX'라 부르지만, 버티고 있어요"
http://v.media.daum.net/v/20171208102101715
['갑질', '공유', '폭언']
{'직장갑질119', '가면무도회', '회사', '피해자들'}
52 LG, "허프 베팅액, 일본에 밀리지 않을 것"
http://v.media.daum.net/v/20171203064439717
['데이비드 허프']
{'허프', '투수', '일본', '재계약', '루키', 'LG', '협상', '야쿠르트', '연봉'}
53 물음표의 한화 선발진, 똑똑한 '넘버3'는 나올까
http://v.media.daum.net/v/20171203150150600
NaN
{'토종', '한화', '선발투수', '외국인 투수', '우완', '자원'}
54 [울산 부산] 이승엽 감독대행, "두 마리 토끼 놓쳤지만 후회 없다"
http://v.media.daum.net/v/20171203155612682
NaN
{'우승', 'FA컵', '울산', '이승엽 감독대행', '부산', '승격'}
55 데헤아 10점 "세계 최고"-린가드 9점 "박지성 같았다"(맨유-아스널)
http://v.media.daum.net/v/20171203064721733
['박지성', '린가드', '데 헤아']
{'ESPN', '라카제트', '아스널', '평점', '기록', '알렉상드르', '헤아', '산체스', '린가드', '외질'}
56 [전일야화] '황금빛' 박시후, 신혜선 품고 유인영에 청혼했다
http://v.media.daum.net/v/20171203070006824
['신혜선', '유인영', '박시후', '장소라', '결혼']
{'장소라', '황금빛', '박시후', '최도경', '유인영 청혼', '서지안', '인생'}
57 여고생 목숨 앗아간 '해지 방어'..통신사 직원 압박 심각
http://v.media.daum.

['박진영', '박소현', '김희철', '데뷔전', '주간아이돌']
{'박소현', '발레', '김희철', '인생 술집', '박진영', '데뷔', '연예계'}
121 [ACL 조추첨] 2018 ACL 조편성 확정..K리그 '험난한 본선'
http://v.media.daum.net/v/20171206175750619
NaN
{'2018', '수원', '유나이티드', '상강', 'ACL', '리그', '삼성', '전북 현대'}
122 조달환 "故김주혁, 결혼 부러워하며 술 사주던 기억 생생해"
http://v.media.daum.net/v/20171201214331076
['김주혁', '결혼', '김생민', '창궐', '연예가중계']
{'결혼', '조달환 김주혁', '연예가중계', 'KBS', '촬영', '배우 조달환'}
123 [배지헌의 브러시백] '박병호 컴백' 넥센 타선, 얼마만큼 무서워졌나
http://v.media.daum.net/v/20171201100155038
['넥센 히어로즈', '박병호']
{'상대 투수', '타자', '박병호', '선수', '라인업', '2017', '주전', '넥센'}
124 [단독] '대세' 장기용, tvN '나의 아저씨' 출연..아이유와 재회
http://v.media.daum.net/v/20171201172651193
['장기용', '나의아저씨', '대세', '고백부부', '이선균']
{'아저씨', '고백 부부', '장기용', '아이유', '출연'}
125 [단독]구하라,논현동빌딩 38억원에 매각.임대수입만 4억원대
http://v.media.daum.net/v/20171205095327726
['구하라', '논현동', '카라', '청담동', '홍수현']
{'시세차익', '수익률', '임대 수익', '구하라', '매각', '논현동 건물', '매입'}
126 '감빵생활' 정경호가 없었다면
http://v.media.daum.net/v/20171201095538792
['정경호', '교도소', '박해수', '신원호', '지호'

189 [단독인터뷰] 김혜선 "4억 체납, 10억 갚고 남은 금액..성실히 갚겠다"
http://v.media.daum.net/v/20171211130751928
['이혼', '개인회생', '명단공개', '구창모', '가수']
{'고액 상습', '김혜선', '개인', '과거', '사기', '상습 체납자', '명단', '금액'}
190 [리폿@이슈] '서영이'→'황금빛', 소현경 마법 통했다
http://v.media.daum.net/v/20171211170032078
['내딸서영이', '소현경작가', '신혜선', '박시후', '닐슨코리아']
{'황금빛 인생', '소현경 작가'}
191 '33년 절친' 우병우-최윤수, 나란히 구속 위기
http://v.media.daum.net/v/20171201152304366
['이슈 · 국정원·군 정치개입 의혹', '우병우', '최윤수', '구속영장']
{'구속영장 청구', '수석', '검찰', '차장'}
192 국방부 "JSA 북한군 13초 사격..귀순자 낙엽 덮어 은폐시도"
http://v.media.daum.net/v/20171201115400610
['이슈 · JSA 초소 북한군 귀순', '북한군', '사격', '귀순자']
{'월경 확인', '군사분계선', 'MDL', '북한군', '국방부', '사격', '귀순자', '식별', '은폐'}
193 분노에 휩싸인 '자중지란' 한국당..여당 도와줬다?
http://v.media.daum.net/v/20171206013300137
['이슈 · 2018 예산안 본회의 통과', '한국당', '본회의', '의원총회']
{'한국당', '의원 총회', '반대', '예산안', '의원들', '본회의장'}
194 김문호-나경민-이병규 경쟁, '국대급' 롯데 외야는 전쟁터
http://v.media.daum.net/v/20171206072906455
['김문호', '이병규', '나경민']
{'손아섭', '민병헌', '이병규', '경쟁', '이대호', '지명타자', '포지션', '외야', '

{'인력', '감소', '하반기', '은행 임직원', '점포'}
248 [동아시안컵] '3-4-3 가동' 한국, 남북전 선발 발표..이재성 출격
http://v.media.daum.net/v/20171212150938432
['이재성']
{'신태용호', '감독', '일본', '연맹', '북한전', '한국', '동아시안컵', '이재성', '2연패'}
249 [비디오머그] 인터뷰 도중 사라진 예비 신랑 류현진, 김기태 KIA 감독 때문?
http://v.media.daum.net/v/20171207182701497
['김기태', '류현진', 'LA 다저스']
{'결혼', '청첩장', '배지현', '인터뷰', '감독', '각오', '예비', '김기태', '류현진'}
250 [UCL POINT] '리버풀도 합류' EPL, 4년 만에 전원 16강..5팀은 최초
http://v.media.daum.net/v/20171207063520188
['리버풀 FC']
{'전원 16강', '16강행', '클럽', '리버풀', 'EPL', 'UCL'}
251 김연아 출연 'SKT 평창 응원캠페인' 논란..조직위 중단요구
http://v.media.daum.net/v/20171207065203432
NaN
{'평창', '마케팅', '중단', '응원 캠페인', 'SK 텔레콤'}
252 반려동물 가정분양 3월부터 안된다
http://v.media.daum.net/v/20171203085816571
['반려동물', '동물보호법', '시행규칙']
{'소규모 생산', '개정안', '분양', '반려 동물', '동물보호법', '등록'}
253 세월호 유족, 文대통령에 감사편지..이철조·김현태 선처 호소
http://v.media.daum.net/v/20171204170844778
['이슈 · 세월호 침몰 사고', '세월호', '이철조', '김현태']
{'유골', '세월호', '부탁', '배려', '현장', '편지', '은화', '가족들', '책임자'}
254 [UFC] 4연승 中 파이터, '스턴건

...

{'사진 송혜교', '흔들린 사진', '송혜교 근황'}
2077 [Oh!쎈 현장] '이혼소송' 홍상수vs아내A씨, 10분만에 끝난 첫 변론기일
http://v.media.daum.net/v/20171215171534424
['변론기일', '홍상수감독', '이혼소송', '김민희', '카메라']
{'변론기일', '이혼', '법정', '감독 홍상수', '변호인', '아내', '법원'}
2078 [★톡톡] 김윤석 "우현, 故 이한열의 운동화 한 짝을 기억하고 있었다"
http://v.media.daum.net/v/20171215175108513
['우현', '운동화', '1987', '박종철', '김의성']
{'이한열 열사', '영화', '분실', '김윤석', '우현', '1987년'}
2079 [직격인터뷰]'미우새' PD "신기한 가나, 샘오취리가 韓 좋아하는 이유 알았다"
http://v.media.daum.net/v/20171215145025299
['토니안', '강남', '신기한']
{'샘오취리', '새끼', '예능', '토니안 강남', '가나', '가족', '한국'}
2080 '런닝맨' 최귀화 "하하의 대학교 후배, 김종국보다 2살 동생"
http://v.media.daum.net/v/20171215104929152
['하하', '최귀화', '범죄도시', '고보결', '이상엽']
{'악역 배우', '영화', '최귀화', '런닝맨', '하하의', '대학교', '후배'}
2081 [리폿@뉴스룸] 손석희도 배웠다..정우성, 친선대사의 품격
http://v.media.daum.net/v/20171215064240322
['손석희', '난민촌', '뉴스룸', '유엔난민기구', '로힝야']
{'친선대사', '정우성', '난민촌', '손석희', '유엔난민기구', '현실', '캠프', '참혹'}
2082 '인생술집' 박나래 "나래바 VVVIP 이서진, 야관문주 한통 비우고 쓰러져"
http://v.media.daum.net/v/20171215065809477
['이서진',

2135 MLB 세인트루이스가 선사한 '인간적인' 트레이드의 품격
http://v.media.daum.net/v/20171215083353793
['세인트루이스 카디널스', '스티븐 피스코티']
{'세인트루이스', '구단', '피스 코티', '오클랜드', '트레이드', '미국'}
2136 손흥민 "맨시티가 무적? 그들도 인간일 뿐..두렵지 않다"
http://v.media.daum.net/v/20171215085806298
['손흥민']
{'맨시티', '경기', '자신감', '15연승', '손흥민', '토트넘', '인간', '축구', 'EPL'}
2137 스페인 언론, "이강인, 실바‧모드리치와 유사해"
http://v.media.daum.net/v/20171215132616741
NaN
{'프로', '발렌시아', '데스마르케', '지난', '한국', '스페인 언론', '후베닐', '이강인'}
2138 [원투펀치 288회 1부]  맨시티는 프리미어리그 최고의 팀이 될 수 있을까?
http://v.media.daum.net/v/20171215142835538
NaN
{'친구 검색', '맨시티', '최고', '역대 최강', '한준희', '원투펀치'}
2139 '급랭' FA시장, 잔류 협상 테이블도 '찬바람' 속 평행선
http://v.media.daum.net/v/20171215060353033
['정근우', '한화 이글스']
{'계약', '옵션', '선수들', '시장 분위기', '협상', 'FA'}
2140 시상식·윈터미팅 종료, 'FA 12명과 김현수'는 어디로
http://v.media.daum.net/v/20171215102948467
['김현수', '필라델피아 필리스']
{'계약', '구단', '잔류', '메이저리그', '선수들', '김현수', 'KBO리그', '시상식', '롯데', 'FA'}
2141 [이성모의 Respect] "정말 힘들었습니다" 석현준은 어떻게 절망을 이겨냈나
http://v.media.daum.net/v/201712151110488

{'당첨번호', '맞힌', '785', '1등 당첨자', '만원씩'}
2198 귤에 붙어 있는 그거 뭐지? 이름 모를 '그거'를 알려주마
http://v.media.daum.net/v/20171216060303227
['귤', '피자', '이란']
{'물건', '귤락', '이름', '사료', '사일리지', '볏단', '곤포', '모양', '클립'}
2199 한국당 광역 6곳 수성? 글쎄.. 여당 서울시장? 아마도..
http://v.media.daum.net/v/20171216030138299
['이슈 · 2018년 6·13 지방선거', '한국당', '지방선거', '서울시장']
{'국민의당', '한국당', '승리', '민주당', '후보', '여권', '지방선거', '문재인', '정당', '광역단체장'}
2200 일본의 청년 파견작전.. 시골서 나무 베고 연봉 4600만원
http://v.media.daum.net/v/20171216034727581
['공무원', '고령화', '특별교부세']
{'정착', '도시', '일본', '지방', '지자체', '마을', '협력 대원', '지역'}
2201 [노컷스토리] 누가 손 교수를 벼랑 끝으로 몰았나?
http://v.media.daum.net/v/20171216060308230
['대자보', '누가', '경주시']
{'학교', '대자보', '성추행', 'B교수', 'A교수', '어머니', '학생', '교수의'}
2202 정부 "北 평창올림픽 참가"..北 "핵보유국으로 평화사랑"(종합)
http://v.media.daum.net/v/20171216114347851
['이슈 · 북한 핵·미사일 도발', '핵보유국', '평창올림픽', '유엔안전보장이사회']
{'공화국 북한', '비핵화', '대사', '유엔 안전보장이사회', '대화', '차관'}
2203 '13월의 세금폭탄' 피하는 연말정산 꿀팁
http://v.media.daum.net/v/20171216170014935
['이슈 · 2017 연말정산', '연말정산', '

{'황금빛', '원양 어선', '눈빛', '미정', '천호진', '신세', '인생'}
2255 '나혼자' PD "박나래♥기안84, 이미 끝난 줄 알았는데..깜짝 놀랐다" [인터뷰]
http://v.media.daum.net/v/20171216102105099
['기안84', '나혼자', '개그우먼박나래', '김충재', '나혼자산다']
{'혼자', '삼각관계', '분위기', '깜짝', '박나래', '기안 84', '형성'}
2256 닉쿤 "韓 활동 뜸하니 은퇴한 줄 알아"(인터뷰①)
http://v.media.daum.net/v/20171216084019145
['세븐', '태국', '팬미팅', '낯선', '아이돌']
{'활동', '닉쿤', '브라더 시스터', '태국 영화', '연기자'}
2257 '세상에서' 원미경 결국 피토했다 "여보!!!나 왜이래!"[종합]
http://v.media.daum.net/v/20171216221814465
['최민호', '유재명', '세상에서가장아름다운이별', '유동근', '최지우']
{'연수', '상태', '병원', '정수', '정철', '엄마', '눈물', '행패', '원미경'}
2258 [종합] '언터처블' 진구, 김성균 구하려다 부상..작전 들켰다
http://v.media.daum.net/v/20171216000732468
['김성균', '진구', '서이라', '박태진', '문신']
{'서는', '서이라', '박태진', '진구', '언터처블', '장범식', '김성균', '장준', '장기', '준서'}
2259 '컬투쇼' 이하늘 "김창렬 중1 아들과 낚시, 울컥했다"
http://v.media.daum.net/v/20171216153416120
['컬투쇼', '두시탈출컬투쇼', '에피소드', 'djdoc']
{'김창렬', '컬투쇼', '이하늘', '중1', '주환 아들'}
2260 이유영, 내일(17일) 故 김주혁 49재 추모미사 동참한다
http://v.media.daum.net/v/20171216141002051
['

ValueError: '맨시티맨시티의' is not in list